# OpenRouter AI: Complete Tutorial

## What is OpenRouter?

OpenRouter provides a unified API that gives you access to hundreds of AI models through a single endpoint, while automatically handling fallbacks and selecting the most cost-effective options. This makes it an ideal solution for developers who want flexibility in model selection without managing multiple API integrations.

### Key Benefits:
- **Unified Interface**: Access 100+ AI models through one API
- **Cost Optimization**: Automatic selection of most cost-effective models
- **Reliability**: Built-in fallbacks when providers go down
- **OpenAI Compatible**: OpenRouter normalizes the schema across models and providers to comply with the OpenAI Chat API

## Setup Requirements

Before starting, you'll need:
1. OpenRouter account and API key from [openrouter.ai](https://openrouter.ai)
2. Python environment with required packages
3. `.env` file for secure API key management

In [ ]:
# Install required packages
!pip install openai python-dotenv requests langchain crewai litellm -q

## Environment Setup

Create a `.env` file in your project directory with your API keys:

In [ ]:
# Load environment variables from .env file
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Retrieve API keys
OPENROUTER_API_KEY = os.getenv('OPENROUTER_API_KEY')
YOUR_SITE_URL = os.getenv('YOUR_SITE_URL', 'https://localhost:3000')
YOUR_SITE_NAME = os.getenv('YOUR_SITE_NAME', 'Tutorial App')

# Verify API key is loaded
if not OPENROUTER_API_KEY:
    print("⚠️  Warning: OPENROUTER_API_KEY not found in environment variables")
    print("Please add OPENROUTER_API_KEY=your_key_here to your .env file")
else:
    print("✅ API key loaded successfully")

## Method 1: Using OpenAI SDK (Recommended)

The easiest way to use OpenRouter is through the OpenAI SDK with modified parameters:

In [ ]:
from openai import OpenAI

# Initialize OpenRouter client
client = OpenAI(
    api_key=OPENROUTER_API_KEY,
    base_url="https://openrouter.ai/api/v1",
    default_headers={
        "HTTP-Referer": YOUR_SITE_URL,
        "X-Title": YOUR_SITE_NAME,
    }
)

# Basic chat completion
def chat_with_openrouter(message, model="openai/gpt-3.5-turbo"):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "user", "content": message}
            ],
            max_tokens=150,
            temperature=0.7
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"

# Test the function
result = chat_with_openrouter("Explain quantum computing in one sentence.")
print("Response:", result)

## Method 2: Direct API Calls

For more control, you can make direct HTTP requests:

In [ ]:
import requests
import json

def direct_api_call(prompt, model="anthropic/claude-3-haiku"):
    url = "https://openrouter.ai/api/v1/chat/completions"
    
    headers = {
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "HTTP-Referer": YOUR_SITE_URL,
        "X-Title": YOUR_SITE_NAME,
        "Content-Type": "application/json"
    }
    
    data = {
        "model": model,
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "max_tokens": 100
    }
    
    try:
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()
        return response.json()['choices'][0]['message']['content']
    except Exception as e:
        return f"Error: {str(e)}"

# Test direct API call
result = direct_api_call("What is machine learning?")
print("Direct API Response:", result)

## Model Comparison and Selection

OpenRouter provides access to various models with different capabilities and costs:

In [ ]:
# Available models (sample)
MODELS = {
    "gpt-3.5": "openai/gpt-3.5-turbo",
    "gpt-4": "openai/gpt-4",
    "claude-3": "anthropic/claude-3-haiku",
    "gemini": "google/gemini-pro",
    "llama": "meta-llama/llama-2-70b-chat"
}

def compare_models(prompt, models_to_test=["gpt-3.5", "claude-3"]):
    results = {}
    
    for model_name in models_to_test:
        if model_name in MODELS:
            print(f"Testing {model_name}...")
            response = chat_with_openrouter(prompt, MODELS[model_name])
            results[model_name] = response
    
    return results

# Compare responses from different models
comparison_results = compare_models("Explain AI in 20 words.")

for model, response in comparison_results.items():
    print(f"\n{model.upper()}:")
    print(response)

## Integration with CrewAI

CrewAI allows you to build teams of AI agents. Here's how to integrate it with OpenRouter:

In [ ]:
from crewai import Agent, Task, Crew
from langchain_openai import ChatOpenAI

# Configure OpenRouter LLM for CrewAI
openrouter_llm = ChatOpenAI(
    api_key=OPENROUTER_API_KEY,
    base_url="https://openrouter.ai/api/v1",
    model="openai/gpt-3.5-turbo",
    temperature=0.7
)

# Create agents
researcher = Agent(
    role='Research Analyst',
    goal='Research and analyze topics thoroughly',
    backstory='Expert researcher with analytical skills',
    llm=openrouter_llm,
    verbose=True
)

writer = Agent(
    role='Content Writer',
    goal='Write engaging and informative content',
    backstory='Professional writer with expertise in simplifying complex topics',
    llm=openrouter_llm,
    verbose=True
)

# Create tasks
research_task = Task(
    description='Research the latest trends in AI and machine learning',
    agent=researcher,
    expected_output='A comprehensive research summary'
)

writing_task = Task(
    description='Write a blog post based on the research findings',
    agent=writer,
    expected_output='A well-written blog post'
)

# Create crew and execute
crew = Crew(
    agents=[researcher, writer],
    tasks=[research_task, writing_task],
    verbose=True
)

print("CrewAI setup complete! Ready to execute collaborative AI tasks.")

## Advanced Features

### Streaming Responses

In [ ]:
def stream_response(message, model="openai/gpt-3.5-turbo"):
    """Stream response from OpenRouter for real-time output"""
    try:
        stream = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": message}],
            stream=True,
            max_tokens=200
        )
        
        print("Streaming response:")
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                print(chunk.choices[0].delta.content, end="", flush=True)
        print("\n\nStream complete!")
        
    except Exception as e:
        print(f"Streaming error: {str(e)}")

# Test streaming
stream_response("Tell me a short story about AI.")

## Best Practices & Tips

1. **Security**: You must protect your API keys and never commit them to public repositories
2. **Error Handling**: Always implement try-catch blocks for API calls
3. **Model Selection**: Choose models based on your specific use case and budget
4. **Rate Limits**: Be aware of rate limits and implement appropriate delays
5. **Cost Monitoring**: Monitor usage through OpenRouter dashboard

### Environment Variables Template (.env file):
```
OPENROUTER_API_KEY=your_openrouter_api_key_here
YOUR_SITE_URL=https://yourapp.com
YOUR_SITE_NAME=Your App Name
```

## Conclusion

OpenRouter simplifies AI model integration by providing a unified interface to hundreds of models. Whether you're building simple chatbots or complex multi-agent systems with CrewAI, OpenRouter offers the flexibility and reliability needed for production applications.

Key takeaways:
- Use environment variables for secure API key management
- Leverage OpenAI SDK compatibility for easy integration
- Experiment with different models to find the best fit
- Implement proper error handling and monitoring

Start building your AI applications with OpenRouter today!